# **SELETOR DE ATIVOS GRUPOS GEOPOLÍTICOS DE PAÍSES E ATIVOS IN LOCO**

In [20]:
import pandas as pd
import os

In [21]:
# Lista completa de países desenvolvidos (baseada no FMI e Banco Mundial)
paises_desenvolvidos = {
    "US", "CA",  # América do Norte
    "AT", "BE", "DK", "FI", "FR", "DE", "IE", "IT", "LU", "NL", "NO", "PT", "ES", "SE", "CH", "GB",  # Europa
    "AU", "HK", "IL", "JP", "KR", "NZ", "SG", "TW"  # Ásia-Pacífico
}

In [22]:
# Mapeamento completo de moeda para região
moeda_para_pais = {
    "USD": "US", "CAD": "CA", "GBP": "GB", "EUR": "FR", "JPY": "JP", "AUD": "AU", "NZD": "NZ",
    "CHF": "CH", "SEK": "SE", "NOK": "NO", "DKK": "DK", "FJD": "FI", "BEF": "BE", "ATS": "AT",
    "IEP": "IE", "SGD": "SG", "KRW": "KR", "ILS": "IL", "CNY": "CN", "INR": "IN", "BRL": "BR",
    "RUB": "RU", "ZAR": "ZA", "MXN": "MX", "ARS": "AR", "TRY": "TR", "HKD": "HK", "TWD": "TW",
    "MYR": "MY", "THB": "TH", "IDR": "ID", "PHP": "PH", "SAR": "SA", "AED": "AE", "VND": "VN",
    "ITL": "IT", "LUF": "LU", "ESP": "ES", "PTE": "PT", "NLG": "NL"
}

In [23]:
def classificar_ativos_por_pais(df, coluna_regiao):
    """
    Classifica os ativos entre países desenvolvidos e em desenvolvimento com base na coluna REGION.
    """
    df["Classificação Econômica"] = df[coluna_regiao].apply(
        lambda x: "Desenvolvido" if x in paises_desenvolvidos else "Em Desenvolvimento"
    )
    return df

def adicionar_inloco(df, coluna_regiao, coluna_moeda):
    """
    Adiciona uma coluna 'inloco' que indica se a moeda é do mesmo país da região.
    """
    df["inloco"] = df.apply(lambda row: 1 if moeda_para_pais.get(row[coluna_moeda]) == row[coluna_regiao] else 0, axis=1)
    return df

In [24]:
# Diretório dos arquivos no Google Colab
data_path = "/content/"

# Lista de arquivos CSV a serem processados
arquivos = {
    "ETFS_AMERICAS": os.path.join(data_path, "ETFS AMERICAS.csv"),
    "ETFS_APAC": os.path.join(data_path, "ETFS APAC.csv"),
    "ETFS_EMEA": os.path.join(data_path, "ETFS EMEA.csv"),
    "STOCKS_APAC": os.path.join(data_path, "STOCKS APAC.csv"),
    "STOCKS_CAN": os.path.join(data_path, "STOCKS CANADA MEXICO.csv"),
    "STOCKS_EMEA": os.path.join(data_path, "STOCKS EMEA.csv")
}

In [25]:
# Processar os arquivos e classificar os ativos
resultados = {}
for nome, caminho in arquivos.items():
    try:
        df = pd.read_csv(caminho, delimiter=';', encoding='utf-8', on_bad_lines='skip')
        if "REGION" in df.columns and "CURRENCY" in df.columns:
            df = classificar_ativos_por_pais(df, "REGION")
            df = adicionar_inloco(df, "REGION", "CURRENCY")
            resultados[nome] = df
    except Exception as e:
        print(f"Erro ao processar {nome}: {e}")

In [26]:
# Salvar os arquivos classificados
for nome, df in resultados.items():
    df.to_csv(os.path.join(data_path, f"{nome}_classificado.csv"), index=False, sep=';')
    print(f"Arquivo salvo: {os.path.join(data_path, f'{nome}_classificado.csv')}")

Arquivo salvo: /content/ETFS_AMERICAS_classificado.csv
Arquivo salvo: /content/ETFS_APAC_classificado.csv
Arquivo salvo: /content/ETFS_EMEA_classificado.csv
Arquivo salvo: /content/STOCKS_APAC_classificado.csv
Arquivo salvo: /content/STOCKS_CAN_classificado.csv
Arquivo salvo: /content/STOCKS_EMEA_classificado.csv


In [27]:
# Criar arquivos filtrados (apenas desenvolvidos e inloco == 1)
for nome, df in resultados.items():
    df_filtrado = df[(df["Classificação Econômica"] == "Desenvolvido") & (df["inloco"] == 1)]
    df_filtrado.to_csv(os.path.join(data_path, f"{nome}_filtrado.csv"), index=False, sep=';')
    print(f"Arquivo filtrado salvo: {os.path.join(data_path, f'{nome}_filtrado.csv')}")

Arquivo filtrado salvo: /content/ETFS_AMERICAS_filtrado.csv
Arquivo filtrado salvo: /content/ETFS_APAC_filtrado.csv
Arquivo filtrado salvo: /content/ETFS_EMEA_filtrado.csv
Arquivo filtrado salvo: /content/STOCKS_APAC_filtrado.csv
Arquivo filtrado salvo: /content/STOCKS_CAN_filtrado.csv
Arquivo filtrado salvo: /content/STOCKS_EMEA_filtrado.csv


In [28]:
# Exibir um exemplo do resultado filtrado
if resultados:
    primeiro_nome, primeiro_df = next(iter(resultados.items()))
    primeiro_df_filtrado = primeiro_df[(primeiro_df["Classificação Econômica"] == "Desenvolvido") & (primeiro_df["inloco"] == 1)]
    print(f"Exemplo de filtro para {primeiro_nome}:")
    print(primeiro_df_filtrado.head())

Exemplo de filtro para ETFS_AMERICAS:
   SYMBOL                   DESCRIPTION IBKR SYMBOL CURRENCY PRODUCT REGION  \
5    9840        SPDR GOLD SHARES - USD        9840      USD     ETF     US   
6     AAA  ALTERNATIVE ACCESS FIRST PRI         AAA      USD     ETF     US   
7    AAAU   GOLDMAN SACHS PHYSICAL GOLD        AAAU      USD     ETF     US   
9    AADR   ADVISORSHARES DORSEY WRIGHT        AADR      USD     ETF     US   
10   AAPB  GRANITESHARES 2X LONG AAPL D        AAPB      USD     ETF     US   

   EXCHANGE *PRIMARY EXCHANGE Classificação Econômica  inloco  
5                       SEHK*            Desenvolvido       1  
6                        NYSE            Desenvolvido       1  
7                        AMEX            Desenvolvido       1  
9                     NYSENAT            Desenvolvido       1  
10                        IEX            Desenvolvido       1  
